In [1]:
from enhanced_lib.exchange import ExchangeCache,Account

account = Account(
    'http://localhost:8000',
    "main_account",
    "sub_account",
)

exchange = ExchangeCache(account, "BTCUSDT")

# await exchange.fetch_latest(False)

# exchange.open_orders.get_tp_orders('long',True)

# await account.fetch_config('BTCUSDT')
await exchange.fetch_latest(True)
# exchange.get_next_tradable_zone("long")
exchange.get_next_tradable_zone("short")
# exchange.get_zone_for_position('long')
# exchange.maximum_sell_size
# exchange.future_instance.config.get_trading_zones('kind')
# exchange.open_orders.get_sl_orders('short',True)
# await account.update_config_fields('BTCUSDT',{'budget':400,'max_size':0.21})
# await account.update_config_resistance('BTCUSDT')
# account.general_config['config']['BTCUSDT']
# exchange.future_instance.config.as_dict()
future_instance = exchange.future_instance

# future_instance.get_trading_zones('long')
# future_instance.config.long_liquidation_price
# future_instance.config.as_dict()
# zones = future_instance.config.get_trading_zones('short')
# zones
# stop = future_instance.config.long_liquidation_price
# stop
# app_config = future_instance.config.app_config
# app_config.entry = 44779.3
# app_config.stop = stop
# # app_config.raw = True
# # workers.determine_optimum_reward(app_config)
# exchange.maximum_sell_size
# exchange.get_zone_for_position('long')
# exchange.get_zone_for_position('long')
# exchange.get_next_tradable_zone('long')
# exchange.config_params_for_future_trades2('long',True)

In [2]:
from enhanced_lib.calculations.shared import build_config
from enhanced_lib.calculations.workers.optimum_risk_reward import eval_func,determine_optimum_reward,run_in_parallel
from enhanced_lib.calculations.workers.utils import chunks_in_threads,run_in_threads
app_config = future_instance.config.app_config
app_config.entry = 41993.9
app_config.stop = 44779.3
app_config.kind = 'short'
app_config.risk_reward = 80
# app_config.risk_per_trade = 70
print('app_config',app_config)
# app_config.support = 41900
# app_config.resistance  = 45000
# signal = build_config(app_config,{
#     # 'raw_instance':True,
#     'decimal_places':app_config.decimal_places,
#     'increase':True,
#     'risk_reward':80,
#     'risk_per_trade': 6,
#     'entry': 44200,
#     'kind': 'long',
#     'stop': 42000,
#     'currentEntry':44200
# })
# signal
# # signal.get_bulk_trade_zones(44200,kind='long')
app_config.strategy = 'entry'
# eval_func(80,app_config)
# for i in range(30,199,1):
#     uu = eval_func(i,app_config)
#     # print('uu',len(uu['result']))
# func = run_in_parallel(eval_func, [(x,app_config) for x in range(30,199,1)],no_of_cpu=8)
# func
# bb = chunks_in_threads(eval_func, [(x,app_config) for x in range(30,199,1)],no_of_cpu=4,num_threads=40)
# len(bb)
# func = run_in_threads(eval_func, [(x,app_config) for x in range(30,199,1)],num_threads=2)
# determine_optimum_reward(app_config)
# exchange.future_instance.config.long_liquidation_price
# exchange.get_next_tradable_zone('long')
# exchange.get_next_tradable_zone('short')
exchange.config_params_for_future_trades('short',True,ignore=True)

app_config AppConfig(fee=0.06, risk_per_trade=6, risk_reward=80, focus=29938, budget=900, support=40272.1, resistance=44779.3, percent_change=0.08, tradeSplit=8, take_profit=None, kind='short', entry=41993.9, stop=44779.3, min_size=0.003, minimum_size=0.003, price_places='%.1f', decimal_places='%.3f', strategy='quantity', as_array=None, raw=None)
func []
highest 0
index -1
error ('No optimum reward found for ', 6)
size for risk 6 0.0
func []
highest 0
index -1
error ('No optimum reward found for ', 7)
size for risk 7 0.0
size for risk 9 0.003
size for risk 12 0.003
size for risk 16 0.009
size for risk 21 0.01
size for risk 27 0.017
size for risk 34 0.023
size for risk 42 0.031
size for risk 51 0.04
size for risk 61 0.048
size for risk 72 0.062
size for risk 84 0.07
size for risk 97 0.077
size for risk 111 0.087
size for risk 126 0.101
size for risk 142 0.111
size for risk 159 0.125
size for risk 177 0.142
size for risk 196 0.158
size for risk 216 0.173
size for risk 237 0.19
size for r

[{'stop': 44779.3,
  'entry': 40272.1,
  'size': 0.4,
  'risk_reward': 30,
  'risk_per_trade': 471,
  'trades': [{'entry': 44867.9,
    'risk': 15.7,
    'quantity': 0.094,
    'sell_price': 40272.1,
    'incurred_sell': 38985.2,
    'stop': 45035.3,
    'pnl': 1342.225,
    'fee': 3.374,
    'net': 758.53,
    'incurred': 552.978,
    'stop_percent': 0.004,
    'rr': 30,
    'avg_entry': 43795.0,
    'avg_size': 0.381,
    'start_entry': 40272.1},
   {'entry': 44701.1,
    'risk': 15.7,
    'quantity': 0.047,
    'sell_price': 40272.1,
    'incurred_sell': 33341.2,
    'stop': 45035.3,
    'pnl': 910.22,
    'fee': 1.681,
    'net': 371.425,
    'incurred': 533.916,
    'stop_percent': 0.007,
    'rr': 30,
    'avg_entry': 43443.6,
    'avg_size': 0.287,
    'start_entry': 40272.1},
   {'entry': 44535.0,
    'risk': 15.7,
    'quantity': 0.031,
    'sell_price': 40272.1,
    'incurred_sell': 27927.5,
    'stop': 45035.3,
    'pnl': 702.048,
    'fee': 1.104,
    'net': 239.833,
    'i